In [6]:
from sklearn.model_selection import train_test_split
from preprocessing import tokenizador_negacion, tokenizador_dummy, tokenizador_snowball_stem,\
                            tokenizador_spacy_lemma, tokenizador_spacy_simple
from cargar_csv import load_data

IndentationError: unexpected indent (<ipython-input-6-3cdbf0f4f657>, line 3)

In [11]:
from nltk.corpus import stopwords
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

In [67]:
import pickle
import os

In [7]:
stop = stopwords.words('spanish')

In [2]:
X, y = load_data(as_tuples = False, procesar_no=False)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

In [13]:
tfidf_pipeline = Pipeline([('vect', TfidfVectorizer()), ('clf', MultinomialNB() )])
param_grid = [{'vect__ngram_range': [(1,1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizador_dummy,
                                   tokenizador_negacion,
                                   tokenizador_snowball_stem,
                                   tokenizador_spacy_lemma],
               }]

gridsearch = GridSearchCV(tfidf_pipeline, param_grid, scoring = 'precision', verbose=1, n_jobs=3)


In [14]:
gridsearch.fit(X_train, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=3)]: Done  24 out of  24 | elapsed: 41.7min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...rue,
        vocabulary=None)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params=None, iid=True, n_jobs=3,
       param_grid=[{'vect__stop_words': [['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', ...okenizador_snowball_stem at 0x7fd377b16d08>, <function tokenizador_spacy_lemma at 0x7fd36ae8abf8>]}],
       pre_dispatch='2*n_jobs', refit=True, return_

In [15]:
print('Best parameter set: ' + str(gridsearch.best_params_))
print('Best precision: {}'.format(gridsearch.best_score_))

Best parameter set: {'vect__stop_words': ['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosostros', 'vosostras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás

In [16]:
clf = gridsearch.best_estimator_
print('Precision in test: {}'.format(clf.score(X_test, y_test)))

Accuracy in test: 0.8228404099560761


In [61]:
padrino_bah = '"El Padrino" me parece un film notable, pero personalmente encuentro que es la película más sobrevalorada. No porque esté muy sobrevalorada, sino porque está sobrevalorada por muchos.\
La habré visto unas cinco veces, y nunca he encontrado ninguna escena, ningún pasaje, ningún diálogo, ninguna actuación, ningún plano, ni ningún momento que haya quedado fascinado. Nada. Es toda la película notable, sin errores, muy bien hecha, pero no hay nada que me haya asombrado, sorprendido, extasiado, golpeado o cualquier otro verbo o adjetivo que asocie con una obra maestra. Siempre lo he buscado y nunca lo he encontrado. Nada.''\
Otra cosa que no trago es la actuación de Marlon Brando. Lo siento, pero siempre que lo veo de Don Corleone veo a alguien con algodones por dentro de los mofletes, y no puedo abstraerme de esa chorrada. No cuela.\
Finalmente, me pasa algo muy curioso al cabo de unos días de ver la película. Me hago siempre la misma pregunta: "¿De qué va "El Padrino"? Y he de hacer un gran esfuerzo para recordarlo.\
Naturalmente si me hacen "una oferta que no pueda rechazar" cambiaré mi crítica.'

In [62]:
padrino_ye = 'En mi humilde opinión, es lo mejor que se ha hecho en toda la historia del Séptimo Arte. El talentoso Mario Puzo encontró un alter ego en Francis Ford Coppola, y ambos se lucieron. No sólo es la historia de la mafia mejor contada, sino que es un verdadero drama familiar, en el que las armas quedan en un segundo plano. Sus personajes son tan humanos, tan reales, que uno termina involucrándose.\
Marlon Brando (ya un inmortal) tuvo la doble fortuna de interpretar el mejor papel de su vida, y de compartir con la crema y nata de la actuación estadounidense: el talentoso Al Pacino, el vibrante James Caan, el portentoso Duvall, las geniales Keaton y Shire, el sutil Cazale.\
De otro lado, el trabajo de fotografía y la excelente banda sonora (elegantemente sombría: nada mejor para la ocasión), junto al inteligente manejo de cámara, son un encanto para el buen cinéfilo.\
Podría decirse que nadie ha vivido enteramente si no ha visto, una vez al menos, este prodigio.'

In [66]:
clf.predict_proba([padrino_bah])[0][1], clf.predict_proba([padrino_ye])[0][1]

(0.5280348886356584, 0.7028066422105792)

#### Exportamos el clasificador/pipeline

In [70]:
pickle.dump(clf, open(os.path.join('models','MultinomialNB_MuchoCine.p'),'wb'))